# Comparing Networks

In this notebook, we want to compare different network to each other.
We will start of with...

- Untrained Model
- Trained Model
- Trained Model (Exact Unlearning)*$^1$

All these model should serve as comparison point for approximate MU Algorithms, which we want to try out

*$^1$ Exact Unlearning: refers to an complete retraining of the Model, without the samples which we want to unlearn.

In [16]:
import os
import sys

# Add the parent directory to sys.path
sys.path.append(os.path.join('..', 'src'))

import torch
from torch.utils.data import DataLoader
import copy
from PIL import Image
import numpy as np
# from helper import get_model TODO Muss überarbeitet werden
from mlp_dataclass import MNIST_CostumDataset, TwoLayerPerceptron
from training import main, train_n_models

EXAMPLE = os.path.join("..", "data", "mnist_backup", "7e", "test_36.png")

USED_DATASET = "mnist"
# OUT OF: ["mnist", "cmnist", "fashion_mnist"]
data = MNIST_CostumDataset(
    test = True,
    dataset_name = USED_DATASET,
    download=True,
)

Data already saved to:  ..\data\mnist


In [17]:
def prepare_sample(path:str):
    """preps sample to pass to model, transforms to tensor, reshape to 784, shaping it as batch"""
    sample = Image.open(path).convert("L")
    sample = torch.Tensor(np.array(sample)).reshape(784)
    
    return sample.unsqueeze(0)

def show_sample(sample:torch.Tensor):
    """Converts the sample to a numpy array and reshapes it to a 28x28 image"""
    sample = sample.reshape(28, 28)
    sample = sample.numpy()
    sample = Image.fromarray(sample)
    return sample

In [18]:
# show_sample(prepare_sample(EXAMPLE)).show()

____

### Untrained Model

We want to include an untrained model as a pour reference point.

Later, we will compare the differently trained (and untrained) models on different metrics. We are thinking, that the **untrained model should always be the furthest neighbor** and see if thats true. Also out of pur curiosity, how do the models compare according to the found metrics to the untrained model.

It could be interesting to see, if the unlearned models tend to be more similar to the "Exact MU Model" or to the "Untrained Model"

The "Trained Model"´s and "Exact Model"´s starting point is the "Untrained Model". That means all models can be traced back to it.

In [19]:
# untrained_model = TwoLayerPerceptron(input_dim=784, output_dim=10)

# torch.save(untrained_model.state_dict(), f"..{os.sep}data{os.sep}models{os.sep}untrained_model")

# print("Shape of Tenosr: ", prepare_sample(EXAMPLE).shape)
# untrained_model(prepare_sample(EXAMPLE))

In [22]:
# Why not intialize n models
# TODO There are no samples in the folder yet
# untrained_ms = {f"untrained_{USED_DATASET}_{v}": TwoLayerPerceptron(input_dim=data.__getitem__(0)[0].shape[1], output_dim=data.__getitem__(0)[0].shape[1]) for v in range(30)}

In [ ]:
# Muss von einem auf Mehrere geändert werden

# untrained_model = TwoLayerPerceptron(input_dim=784, output_dim=10)
# untrained_model.load_state_dict(torch.load(f"..{os.sep}data{os.sep}models{os.sep}untrained_model", weights_only=True))
# untrained_model.eval()

TwoLayerPerceptron(
  (fc1): Linear(in_features=784, out_features=800, bias=True)
  (fc3): Linear(in_features=800, out_features=10, bias=True)
)

In [ ]:
# Muss von einem auf Mehrere geändert werden

# print("Shape of Tenosr: ", prepare_sample(EXAMPLE).shape)
# print(untrained_model(prepare_sample(EXAMPLE)))
# torch.max(untrained_model(prepare_sample(EXAMPLE)), 1)

Shape of Tenosr:  torch.Size([1, 784])
tensor([[-13.3623, -15.0116, -33.1536,  -0.3706, -18.2124, -13.9797, -20.4261,
          -1.6220,  -2.1874, -26.4755]], grad_fn=<LogSoftmaxBackward0>)


torch.return_types.max(
values=tensor([-0.3706], grad_fn=<MaxBackward0>),
indices=tensor([3]))

___

### Trained Model

This model will be the starting point for all MU algorithms. The more it is important that it does not change throughout the experiments, otherwise we need to run everything again.

Notes Regarding the Dataloader: We included Up-/Downsampling. There is a counter in the Dataset class, which controls, which class is next to provide a sample. From the available pool a sample will be chosen at random.

Reason: The amount of available samples should not influence the performance of the model on certain classes.

Regarding the samples for the class "7": There are more samples of sevens without a middle line than ones with.

Ratio: 
- 6268 (train: 5385, test: 883)
- 1025 (train: 880,  test: 145)

Should there be concerns regarding that the bigger part of the sevens (without middle line) will be unlearning?

In [ ]:
# Train a single model first
main(
    new_name="trained_showcase",
    model=None,
    sampling_mode="all",
    balanced_sampling=True,
    dataset_name=USED_DATASET,
    include_val=True,
    logs=True,
)

In [ ]:
trained_ms = train_n_models(
    sampling_mode="except_erased",
    dataset_name=USED_DATASET,
    logs=False,
)

Epoch [1/10] - Train Loss: 10.2791 - Val Loss: 3.7059 - Train Accuracy: 0.7190 - Val Accuracy: 0.8291
                                                                   
Epoch [2/10] - Train Loss: 3.1310 - Val Loss: 1.9731 - Train Accuracy: 0.8250 - Val Accuracy: 0.8330
                                                                   
Epoch [3/10] - Train Loss: 1.9117 - Val Loss: 1.6126 - Train Accuracy: 0.8360 - Val Accuracy: 0.8395
                                                                   
Epoch [4/10] - Train Loss: 1.2980 - Val Loss: 1.4563 - Train Accuracy: 0.8530 - Val Accuracy: 0.8354
                                                                   
Epoch [5/10] - Train Loss: 0.9279 - Val Loss: 0.9853 - Train Accuracy: 0.8780 - Val Accuracy: 0.8581
                                                                   
Epoch [6/10] - Train Loss: 0.8566 - Val Loss: 0.5171 - Train Accuracy: 0.8880 - Val Accuracy: 0.9131
                                                                   
Epoch [7/10] - Train Loss: 0.6793 - Val Loss: 0.4452 - Train Accuracy: 0.9000 - Val Accuracy: 0.9159
                                                                   
Epoch [8/10] - Train Loss: 0.3773 - Val Loss: 0.3985 - Train Accuracy: 0.9260 - Val Accuracy: 0.9213
                                                                   
Epoch [9/10] - Train Loss: 0.2953 - Val Loss: 0.3818 - Train Accuracy: 0.9430 - Val Accuracy: 0.9239
                                                                   
Epoch [10/10] - Train Loss: 0.3581 - Val Loss: 0.3543 - Train Accuracy: 0.9140 - Val Accuracy: 0.9273

Model saved to:  ..\data\models\all\TwoLayerPerceptron_b_trained_model

In [ ]:
# TODO Muss von einem auf Mehrere geändert werden

# trained_model = get_model("trained")

___

### Exact Unlearned Model

This model has the same training conditions as the "Trained Model", with the only difference being, that it had no sevens __with no middle line__

In [ ]:
# Train a single model first
main(
    new_name="trained_showcase",
    model=None,
    sampling_mode="except_erased",
    balanced_sampling=True,
    dataset_name=USED_DATASET,
    include_val=True,
    logs=True,
)

In [ ]:
exact_ms = train_n_models(
    sampling_mode="except_erased",
    dataset_name=USED_DATASET,
    logs=False,
)

Epoch [1/10] - Train Loss: 11.0494 - Val Loss: 3.9705 - Train Accuracy: 0.6790 - Val Accuracy: 0.8060
                                                                   
Epoch [2/10] - Train Loss: 2.8843 - Val Loss: 2.0364 - Train Accuracy: 0.8210 - Val Accuracy: 0.8452
                                                                   
Epoch [3/10] - Train Loss: 1.9582 - Val Loss: 1.9326 - Train Accuracy: 0.8260 - Val Accuracy: 0.8066
                                                                   
Epoch [4/10] - Train Loss: 1.0714 - Val Loss: 0.6197 - Train Accuracy: 0.8630 - Val Accuracy: 0.9021
                                                                   
Epoch [5/10] - Train Loss: 0.9799 - Val Loss: 0.7909 - Train Accuracy: 0.8540 - Val Accuracy: 0.8872
                                                                   
Epoch [6/10] - Train Loss: 0.5125 - Val Loss: 0.5123 - Train Accuracy: 0.9140 - Val Accuracy: 0.9135
                                                                   
Epoch [7/10] - Train Loss: 0.3382 - Val Loss: 0.4401 - Train Accuracy: 0.9360 - Val Accuracy: 0.9180
                                                                   
Epoch [8/10] - Train Loss: 0.6186 - Val Loss: 0.3889 - Train Accuracy: 0.8990 - Val Accuracy: 0.9229
                                                                   
Epoch [9/10] - Train Loss: 0.4423 - Val Loss: 0.3394 - Train Accuracy: 0.9170 - Val Accuracy: 0.9277
                                                                   
Epoch [10/10] - Train Loss: 0.4092 - Val Loss: 0.3211 - Train Accuracy: 0.9370 - Val Accuracy: 0.9291

Model saved to:  ..\data\models\except_erased\TwoLayerPerceptron_b_exact_model

In [ ]:
# TODO Muss von einem auf Mehrere geändert werden

# exact_model = get_model("exact")

___

## Metrics

In [23]:
from metrics import calc_accuracy
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt

### L2 Norm

Compare how the different models are from each other using the L2 Norm, comparing every parameter with each other

In [ ]:
from metrics import model_l2_norm_difference

#### Unlearn to ...

In [ ]:
# ...trained model
a11 = model_l2_norm_difference(untrained_model, trained_model)
sum(a11.values())

In [ ]:
# ...exact model
a12 = model_l2_norm_difference(untrained_model, exact_model)
sum(a12.values())

#### Trained to ...

In [ ]:
# ...untrained model
a21 = model_l2_norm_difference(trained_model, untrained_model)
sum(a21.values())

In [ ]:
# ...exact model
a22 = model_l2_norm_difference(trained_model, exact_model)
sum(a22.values())

#### Exact to ...

In [ ]:
# ...untrained model
model_l2_norm_difference(exact_model, untrained_model)

In [ ]:
# ...trained model
model_l2_norm_difference(exact_model, trained_model)

### KL-Divergence

Used to compare the difference between distribution, but may also be used to compare models by their predicted target distributions.

In [24]:
from metrics import kl_divergence_between_models

#### Unlearned to...

#### Trained to...

#### Exact to...

### Accuracy

- $D_{Gesamt}$
- $D_{Erased}$
- $D_{Remain}$
- $D_{Accuracy Per Class}$

Stop Criterium von 10 nicht fair, da es sich implizit 

- Wie verhält sich das Unlearning über die Anzahl der Epochen?

- Man sollte sich den Loss als Referenz anschauen

- Festes Downsampling, wenn die 7 reduzierte Samples haben, müssten auch die anderen Klassen weniger Samples in der Grundmenge haben.

- auf einen weiteren Datensatz soll es auch getestet werden. (C_MNIST wie robust ist der Algorithmus gegenüber Rausche)
    - ggf. auf weiterem Datensatz ein Merkmal versuchen zu Unlearnen

In [ ]:
from visualize import boxplotting_multimodel_eval

Now, let us see what metrics each model has

In [ ]:
# Macht vllt ned so viel Sinn, fand i nur witzig
# boxplotting_multimodel_eval

In [ ]:
boxplotting_multimodel_eval(trained_ms, dataset_name=USED_DATASET, evaluation="accuracy", logs=False)

In [ ]:
boxplotting_multimodel_eval(exact_ms, dataset_name=USED_DATASET, evaluation="accuracy", logs=False)